In [ ]:
# Achei bem interessante a abordagem.
# Faltou um vinculo entre bicicleta e cliente. Além de não ser armazenado em nenhum lugar os dados digitados. Poderia ter usado uma lista para armazenar tudo que foi cadastrado.
# Vi também que é feito uma checagem de dados. Uma vez feito esse tipo de validação, o ideal é parar a execução uma vez que o dado é inválido. Rodando o código, se eu digitar um email inválido, o programa segue normalmente.
# E as fotos? Isso não foi pensado? Sei que não temos como trabalhar com isso, mas a ideia aqui era mais mostrar a organização do que o código pronto em si. Então coloquem isso na estrutura de vocês, ok?
# No mais, parabéns pelo que foi mostrado. É um bom começo para organizar esse tipo de programa.

In [9]:
#━━━━━━━━━❮Bibliotecas❯━━━━━━━━━
from datetime import datetime
import sys
import re
import urllib.request
import cv2
import numpy as np

# import pytesseract  #ele não instala corretamente, usar instalador para windows https://github.com/UB-Mannheim/tesseract/wiki
                    #instalado em C:\Program Files\Tesseract-OCR
#━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━




In [16]:
import cv2
import numpy as np

# Passo 1: Carregar as imagens de treinamento
positives = ["bike.jpg", "bike2.jpg", "bike3.jpg"]
negatives = ["nbike.jpg", "nbike2.jpg", "nbike3.jpg"]

# Passo 2: Extrair características HOG das imagens
hog = cv2.HOGDescriptor()
features = []
labels = []

for image_path in positives + negatives:
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features.append(hog.compute(gray))
    if "background" in image_path:
        labels.append(0)  # imagem sem objeto
    else:
        labels.append(1)  # imagem com objeto

# Passo 3: Treinar o classificador SVM
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))

features = np.array(features).squeeze()
labels = np.array(labels)
svm.train(features, cv2.ml.ROW_SAMPLE, labels)

# Passo 4: Carregar a imagem de teste
test_image = cv2.imread("test_bike_image.jpg")
gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

# Passo 5: Extrair características da imagem de teste
test_features = hog.compute(gray).squeeze()

# Passo 6: Prever o objeto na imagem de teste
result = svm.predict(test_features.reshape(1, -1))[1]

if result == 1:
    print("Objeto detectado!")
else:
    print("Objeto não detectado.")

C:\Users\pytho\AppData\Local\Temp\ipykernel_27248\2824571787.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.array(features).squeeze()


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'train'
> Overload resolution failed:
>  - ml_StatModel.train() takes at most 2 arguments (3 given)
>  - samples data type = 17 is not supported
>  - Expected Ptr<cv::UMat> for argument 'samples'


In [ ]:
# usar funções lambda

ano_atual = datetime.now().year

def mostrar_menu():
    print('━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━')
    print('1. Cadastrar bike ')
    print('2. Cadastrar cliente')
    print('3. Sair')
    print('━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━')

#Cadastro de bikes
def cadastro_bike():
    marca = input('Digite a marca da bike: ')
    print('')

    modelo = input('Digite o modelo da bike: ')
    print('')

    ano = int(input('Digite o ano de fabricação da bicicleta (entre 1900 e {}):'.format(ano_atual)))
    print('')

    if ano < 1900 or ano > ano_atual:
        print('Ano de fabricação inválido. Tente novamente.')
        print('')
        print('Por conta disso, código será interrompido')
        sys.exit(0)
    else:
         pass
        
    preco = float(input('Digite o preço da bike: '))
    print('')

    bicicleta = {
        'marca': marca,
        'modelo': modelo,
        'ano_fabricacao': ano,
        'preco': preco
    }

    print('Bicicleta cadastrada:')
    print('Marca:', bicicleta['marca'])
    print('')

    print('Modelo:', bicicleta['modelo'])
    print('')

    print('Ano de Fabricação:', bicicleta['ano_fabricacao'])
    print('')
    print('Preço: R$', bicicleta['preco'])



regex = '^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$'    
def check(email):  
    if(re.search(regex,email)):  
        print("Email Verificado com Sucesso")  
          
    else:  
        print("Email Invalido")  
        print('Por conta disso, código será interrompido')
        sys.exit(0)


# cadas
def cadastro_cliente():
    nome = input('Digite o nome do cliente: ')
    print('')

    sexo = input('Digite o sexo do cliente: ')
    print('')
        
    cpf = int(input('Digite o cpf do cliente: '))
    
    email =input('digite o email do cliente')
    check(email)
    print('')

    telefone = input('Digite o telefone do cliente: ')
    print('')

    ano_nas = int(input('Digite o ano de nascimento do cliente: '))
    print('')
    idade = ano_atual - ano_nas

    cep =input('digite o cep do cliente')
    print('')

    cliente = {
        'nome': nome,
        'sexo': sexo,
        'cpf': cpf,
        'email': email,
        'telefone': telefone,
        'idade': idade,
        'cep': cep
    }

    print('Cliente cadastrado:')
    print('Nome:', cliente['nome'])
    print('Sexo:', cliente['sexo'])
    print('CPF:', cliente['cpf'])
    print('Email:', cliente['email'])
    print('Telefone:', cliente['telefone'])
    print('Idade:', cliente['idade'])
    print('CEP:', cliente['cep'])











 # Loop para executar o programa até que o usuário escolha sair
while True:
    mostrar_menu()
    escolha = input('Escolha uma opção: ')
    
    if escolha == '1':
        cadastro_bike()

    elif escolha == '2':
        cadastro_cliente()

    elif escolha == '3':
        print('Obrigado por usar nosso programa de cadastro de bicicletas em Python. Até mais!')
        break

    else:
        print('Opção inválida. Por favor, escolha uma opção válida.')

#Code by Mhb


# Exibindo o dicionário